# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦代表着天、强、积极、刚毅、力量、行动、刚健等含义。

乾卦由六个阳爻（表示积极、刚毅、力量）组成，两个阴爻（表示柔顺、适应、容纳）组成。阳爻在爻的上面，阴爻在爻的下面，这种排列方式被称为“奇数排列”。

乾卦的变化规律是：初爻为阳，二爻为阳，三爻为阳，四爻为阳，五爻为阳，六爻为阳。这种阳爻逐渐增加的过程，象征着事物的发展和壮大。

在《易经》中，乾卦经常与坤卦（代表地）相对应，象征着天地之间的关系。乾卦代表天，象征阳、刚、积极等，而坤卦代表地，象征阴、顺、容纳等。两者相互对应，共同构成了天地之间的相互作用和影响。

乾卦在实际应用中，提醒人们要有积极向上的心态，勇敢面对困难和挑战，采取行动去实现自己的目标和愿望。同时，也要注意保持内心的柔顺和容纳，以便更好地与周围的人和事物相处。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的第62卦，也是八卦之一。师卦代表着师、众、领导、智慧、军队等含义。

师卦由两个阴爻和一个阳爻组成，阳爻在爻的上面，阴爻在爻的下面，这种排列方式被称为“偶数排列”。阴爻表示众、柔、适应、谦虚等，阳爻表示领导、刚、智慧、力量等。

师卦的变化规律是：初爻为阴，二爻为阴，三爻为阳，四爻为阴，五爻为阳，六爻为阴。这种阴爻逐渐增加的过程，象征着事物的积累和发展。

在《易经》中，师卦经常与坤卦（代表地）相对应，象征着天地之间的关系。师卦代表众、柔、适应等，而坤卦代表地、阴、顺、容纳等。两者相互对应，共同构成了天地之间的相互作用和影响。

师卦在实际应用中，提醒人们要有领导能力和智慧，能够组织和协调众人，共同完成艰巨的任务。同时，也要注意保持谦虚和适应，以便更好地与周围的人和事物相处。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，也是六十四卦中的第一卦。乾卦是由两个阳爻夹一个阴爻构成，象征着天，反映了宇宙的生成和运行。

乾卦的卦象是：
上爻（乾），阳爻
中爻（坤），阳爻
下爻（震），阳爻

乾卦的卦辞：
“元”，表示开始；
“贞”，表示正。

乾卦卦辞：
“元”，吉，亨，利，西南 direction。

“利”：“利”是指“有利”，即“有利”之情；“西南”是指“西南方向”。

乾卦的卦象代表天，意味着刚健、强盛、高远，它的特点是对“阳”的追忆和向往。乾卦强劲刚健，它提醒我们把握机会，把握时势，追求成功的最佳状态。

乾卦的卦辞“元”，表示开始，意味着初始，追求初始的最佳状态，获得成功。乾卦的“利”，表示有利，追求利最佳状态，获得成功。

乾卦它提醒我们：把握时机，追求时势，才能获得成功。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着天，也象征着刚强和坚毅。乾卦的卦辞是“元，亨，利，贞”，意味着“ beginning, success, profit, integrity”。它表示万事万物的本源和原理，是成功的基础。乾卦 also means "the heaven", and it is a symbol of the universe, with six lines of yin and one line of yang. It represents the power and strength of the world, and is a symbol of integrity.


In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
师卦（师卦）：师 贞 丈 1 报 贞。
师卦为乾为坤为旅。师卦象征军队，坤象征地，乾象征天。师卦象征军队，军队行于地，宜顺行，不宜逆行，的形象雷。乾为天，为师，象征天，乾为天，为师，象征天，肯任人，宜顺从，不宜逆从。
师卦的时宜：西南 西南 西南
师卦的评断：西南 西南 西南
```
说明：西南为坤，为师，为天空，宜顺从；西南为坤，为师，为天地，不宜逆从。

```
师卦原文：师，贞，丈，1，报，贞。

白话文解释：师卦象征军队，坤象征地，乾象征天。天意示示，顺应天意，行路才会顺利。1卦为阳，坤为阴，合为阴阳和谐。

北宋易学家邵雍解释：天行健，地生风，君子行事应如天行健，顺从事理。

断卦：师卦为乾为坤为旅，象征军队，宜顺行，不宜逆行，象征天意，顺从天理。

犯师：师卦象征军队，不宜逆行，象征天意，应顺应天理。

师卦：师，贞，丈，1，报，贞。

象辞：师卦上坤下乾，坤为地，生风，乾为天，天 compels us to follow the natural order.

断卦：师卦坤下乾上，象征军队，宜顺从，不宜逆行，象征天意。

犯卦：宜顺从天意，遵循自然。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 师卦（师卦）是由《易经》中的师卦和师卦变卦而组成的8卦，师卦为动卦，为兵，为兵者，动而占先机。师卦变卦由师卦变而来，师卦变卦包括：师变卦、师占卦、师吉卦、师凶卦。师卦的原卦为师，下卦为坤，上卦为乾，乾为刚，坤为顺，刚顺相生，占先机之利，先吉后凶。变卦后，变为占师吉凶之卦，得胜之后之象，故为师占卦，师吉卦。 师卦是占卦之一，为兵法卦，占兵法之先机，军旅之利运。师卦是《易经》六十四卦中的第三卦，下坤上乾，乾刚顺，坤顺承，刚顺相生，先吉后凶。师卦宜阳不宜阴，适宜顺从事理，初吉后凶，必须谨慎。 师卦提示：兵者先动，先占先利，把握先机之胜，预吉凶事。


In [13]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》hexagram（卦象）之一，卦象上下一阳一阴，阴爻在阳爻之上，阴柔不能胜阳，于是争竞诉讼，体现了一种不能和谐相处的状态。讼卦象征诉讼争端，同时也提醒人们要合理处理矛盾和纠纷，避免争端。

讼卦的卦象是：下卦为阴，上卦为阳。

卦象解释：讼卦卦象上下一阳一阴，阴爻在阳爻之上，表示虽然阴爻处于优势地位，但由于阳刚的力量，阴不能胜阳，于是出现争竞诉讼的情况。

卦象意义：讼卦提醒人们要合理处理矛盾和纠纷，避免争端。它告诉我们要以和为贵，以和平共处为原则，才能保持和谐顺利。

基本卦象：阴（下）+阳（上）=阴胜阳负，表示争端，诉讼，矛盾。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个乾卦叠加而成，象征着天地之间的相互支撑和互相照耀。在周易中，乾卦代表天，反映刚健强劲的特性。同时，乾卦也代表君子，具有刚健强劲的品德，以及天命般的运行规律。

乾卦的卦辞是：“元、亨、利、贞”，意味着元（ start）亨（顺利）利（吉祥）贞（正），象征着事物天命般的运行规律。同时，乾卦也表达了敬天保元、顺从天意的寓意。

在周易中，乾卦具有以下特性和意义：

1. 象征天：乾卦代表天，象征着刚健强劲的特性。

2. 刚健强劲：乾卦由两个乾卦叠加而成，象征着天地之间的相互支撑和互相照耀。反映了刚健强劲的特性。

3. 天地相互支援：在周易中，乾卦代表天，反映了天命般的运行规律。

4. 敬天保元：乾卦的卦辞中提到“元、亨、利、贞”，表达了敬天保元、顺从天意的寓意。

5. 吉祥：乾卦象征着吉祥如意，代表着天命上的正统和合法。

6. 刚健生动：乾卦具有刚健生动的特质，代表着君子刚健的品德。

7. 刚直不阿：乾卦反映刚健强劲的品德，象征着君子的刚直不阿。

8. 天地之间：在周易中，乾卦代表天，象征着君子应该效法天的运行规律，努力刚健刚直，实现天命般的运行。

9. 刚健强劲：乾卦的特性是刚健强劲，代表着不阿不屈不妥协的品质。

总结起来，乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它象征着天地之间的相互支撑和互相照耀，表达了敬天保元、顺从天意的寓意。同时，乾卦也代表君子，具有刚健强劲的品德，以及天命般的运行规律。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个卦相组合而成，一个是乾卦，另一个是坤卦。乾卦的排列是：元、亨、利、贞，代表天象运行，具有刚健强劲的特性。

在周易中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得，丽（美丽）预示着天将利市，利（吉祥）预示着天将得利。

乾卦的哲学内涵是：天行健，君子以自强不息。它强调的是刚健强劲的品质，鼓励人们不断自我提高，自我完善。

在周易中，乾卦代表的是天，具有刚健强劲的特性。它预示着天将健行，云层将有所得，丽将利市，利将得利。

在周易的哲学中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得

In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由两个卦象组合而成：上卦为坤，代表大地，代表顺从、顺应天道的人；下卦为坎，代表水，代表变化、灵活。

地水师卦的核心卦象是“地水相成”，代表水在地下流动，大地吸收水分，象征着顺利融合双方的力量，代表着顺利和谐地处理双方的矛盾。

在占卜时，地水师卦预示着会遇到困难，必须 flexibility（灵活），求变，代表变通，求变则通。代表预测：困难已至，宜速变通，宜谨慎行事。

地水师卦的核心哲学是：调料需调和使用， wealth（财富）需谨慎使用，二者必需相互配合。

地水师卦适用于占卜、预测未来的情况，提醒人们遇到困难和变故，必须谨慎处理，变通求通。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表地下的水，象征润泽大地的力量。根据《易经》中的解释，地水师卦预示着吉祥如意，具有强大的力量，可以克服各种困难。

根据传统解卦，地水师卦分为两个卦象，第一个卦象是坎卦（水），代表灵活变化的力量，具有吉祥如意，靠智慧取悦于人；第二个卦象是坤卦（地），代表刚健柔顺，具有强大的力量，可以克服各种困难。

地水师卦的核心理念是：吉祥如意，靠智慧取悦于人，靠刚健柔顺的力量，可以克服各种困难。


In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坤（地）和下卦乾（天）组成，象征着天地的相争，具有象征意义。在卜筮时，先提出天象，再详观地象，以确定双方的态度，从而决定是否彼此相争。同时，对方是否愿意主动求胜？在决策时，要考虑利弊得失，同时要考虑双方的意愿，双方都希望得到想要的东西。

讼卦的核心哲学是：天地的相争，双方都需要谨慎处理，通过协商和智慧，以达成共识。在实际应用中，讼卦提醒我们在面对争端和冲突时，要冷静分析，以ilegitimous（合理）的方式解决争端，以避免冲突进一步升级。


ChatGLM3-6B微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易卦象中的一卦，由上卦坎（水）和下卦乾（天）组成，象征着天神下降于坎水之中，具有刚健刚毅之象。讼卦的卦象具有云雾聚集、天光破云的意象，象征着事情将会顺利发展，宜早做准备。

讼卦的时运是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 四爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的解卦是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的卦象和时运都预示着刚健刚毅的特性，天神下降于坎水之中，具有刚健刚毅之象。在事业和生活中遇到困难和挑战时，需要积极应对，宜早做准备，不宜冒险，切勿冒进。在处理问题和决策时，要冷静分析，不宜草率行事，宜等待时机的到来。同时，要注重刚健刚毅的态度，积极应对各种困难和挑战，从而获得成功。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [6]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240118_164514"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [8]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [9]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，由两个阳爻夹一个阴爻构成，象征着天。它代表着刚健、健行、刚健不屈的意义。乾卦象征刚健、健行、坚毅的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在年第uck博士的风水flexibility中，乾卦也象征着天，代表着刚健、健行、坚毅的性格，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风水学中，乾卦代表天，象征着坚毅、刚健、不屈不挠的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在的风水flexibility中，乾卦也象征着天，代表着坚毅、健行、坚毅的性格，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风水学中，乾卦代表天，象征着坚毅、健行、不屈不挠的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [10]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
{'name': '其主要含义是占卜土地和水源的适宜与否，以及它们对人类生活的影响。在占卜中，地水师卦代表一种吉祥的卦象，预示着吉利和繁荣。', 'content': '\n在事业方面，地水师卦预示着事业发展顺利，提示要谨慎行事，行事光明磊落，的字面意思与水有关，表明大吉。\n\n在经商方面，虽然是吉利卦象，但需要谨慎行事，特别是在 business 交往中要谨慎行事，不能轻信他人，防止煮熟的鱼喂鸟，预防意外事件，不能贪图眼前利益，必须谨慎行事。\n\n在求名方面，提示要诚实、勤奋学习，努力提高自己的能力，与世无争，自然成功。\n\n在决策方面，提示要谨慎行事，不能草率行事，必须谨慎研究，根据实际情况制定计划，不能贪图眼前利益而忽视长远利益。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 卦名：地水师卦是一个由三个卦相组合而成，代表着自然界的力量和人类社会的活动。第一个卦是震卦，代表着震动和雷，第二个卦是坤卦，代表着顺从和大地，第三个卦是兑卦，代表着欢愉和快乐。地水师卦的核心哲学是：自然界的力量和人类社会的活动相互影响，人们应该顺应自然，发挥自己的才能，以达到和谐共处的境界。

卦象：震卦为雷，卦象是地震，代表着震动和雷。兑卦为泽，卦象是湖泊，代表着欢愉和快乐。坤卦为地，卦象是坤卦，代表着顺从和大地。

卦象解释：地水师卦的核心哲学是：自然界的力量和人类社会的活动相互影响，人们应该顺应自然，发挥自己的才能，以达到和谐共处的境界。

君子观象：君子应当学习震卦的震动和兑卦的快乐，从而返回震卦的震动。


In [11]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。在周易中，坎代表着险恶，而乾则象征着高洁。这一卦象预示着天地之间的险恶已经来到人们中间，但天依然高悬，没有改变。对于创作者来说，这一卦象提醒了的风险和挑战，然而也给了创作者展望未来的信心。

讼卦的核心哲学是：险象环生，但仍有希望。和高洁的乾卦相比，坎卦代表着险恶，但高洁的乾卦仍然在上面，意味着高洁之志未尝失。因此，尽管讼卦充满风险和挑战，但仍有希望。

对于创作者来说，尽管面临各种风险和挑战，但仍有希望。但需高洁的品质，坚定信念，克服困难，但也需谨慎行事，避免险恶之险。
